In [1]:
import numpy as np
import gym
from gym import Env, spaces, utils
import pygame
from typing import Optional
from io import StringIO
from contextlib import closing
from gym.spaces import Discrete
from gym.wrappers import FlattenObservation
import os

In [2]:
from env_2048 import *

In [4]:
env = BlockDoubleEnv()
env.reset()
env._boardstate = [[4, 8, 4, 2],
  [4, 2, 4, 8],
  [2, 4, 8, 4],
  [2, 4, 8, 2]]
new_state, reward, done, info = env.step(3)
print(env.render())

[8.0, 8.0, 8.0, 2.0]
[4.0, 2.0, 16.0, 8.0]
[0.0, 8.0, 0.0, 4.0]
[0.0, 2.0, 0.0, 2.0]



In [6]:
from IPython.display import clear_output
env = BlockDoubleEnv()
env.reset()  # reset enviornment to default state
env._boardstate = np.array([[4, 8, 4, 2],
  [4, 2, 4, 8],
  [2, 4, 8, 4],
  [2, 4, 8, 2]])
score = 0
while True:
    check = 0
    print(env.render())
    key = input("w,a,s,d\n").lower()
    if key == "w":
        action = 3
    elif key == "d":
        action = 0
    elif key == "s":
        action = 1
    elif key == "a":
        action = 2
    else:
        break
    clear_output()
    new_state, reward, done, info = env.step(action)

[0.0, 0.0, 0.0, 2.0]
[8.0, 8.0, 2.0, 8.0]
[4.0, 2.0, 8.0, 4.0]
[2.0, 8.0, 16.0, 2.0]



In [8]:
env = BlockDoubleEnv()
env.reset()  # reset enviornment to default state
env._boardstate = np.array([[4, 8, 4, 2],
  [4, 2, 4, 8],
  [2, 4, 8, 4],
  [2, 4, 8, 2]])
new_state, reward, done, info = env.step(3)
print(env.render())
new_state, reward, done, info = env.step(3)
print(env.render())
new_state, reward, done, info = env.step(3)
print(env.render())

[8.0, 8.0, 8.0, 2.0]
[4.0, 2.0, 16.0, 8.0]
[0.0, 8.0, 0.0, 4.0]
[0.0, 0.0, 2.0, 2.0]

[8.0, 8.0, 8.0, 2.0]
[4.0, 2.0, 16.0, 8.0]
[0.0, 8.0, 2.0, 4.0]
[0.0, 0.0, 2.0, 2.0]

[8.0, 8.0, 8.0, 2.0]
[4.0, 2.0, 16.0, 8.0]
[4.0, 8.0, 4.0, 4.0]
[0.0, 0.0, 0.0, 2.0]



In [ ]:
import time
turns = 2
env = BlockDoubleEnv()
for turn in range(1,turns+1):
    state = env.reset()
#     env._boardstate = np.array([[4.0, 16.0, 2.0, 8.0],
# [8.0, 4.0, 16.0, 2.0],
# [2.0, 4.0, 4.0, 8.0],
# [0.0, 0.0, 8.0, 2.0]])
    
    done = False
    score = 0

    while not done:
        #clear_output()
        print(env.render())
        action = env.action_space.sample()
        new_state, reward, done, info = env.step(action)
        score += reward
        print(reward)
    print(f"Episode:{turn}, Score:{score}")


In [3]:
from env_2048 import *

env = BlockDoubleEnv()

In [14]:
from keras.models import Sequential

In [20]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Flatten, Rescaling, InputLayer
from keras.optimizers import Adam

states = gym.spaces.utils.flatdim(env.observation_space)
#states = env.observation_space.shape
actions = env.action_space.n

def build_model(states, actions):
    model = Sequential()
    model.add(InputLayer(input_shape=states))
    model.add(Rescaling(scale=1./2048))
    model.add(Flatten())
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [21]:
model = build_model(states, actions)

In [22]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_3 (Rescaling)     (None, 16)                0         
                                                                 
 flatten_3 (Flatten)         (None, 16)                0         
                                                                 
 dense_9 (Dense)             (None, 24)                408       
                                                                 
 dense_10 (Dense)            (None, 24)                600       
                                                                 
 dense_11 (Dense)            (None, 4)                 100       
                                                                 
Total params: 1,108
Trainable params: 1,108
Non-trainable params: 0
_________________________________________________________________


In [23]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn


In [24]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)


ValueError: Error when checking input: expected input_3 to have 2 dimensions, but got array with shape (1, 1, 4, 4)

In [ ]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))
_ = dqn.test(env, nb_episodes=15, visualize=True)